Let's optimise our nutrition and grocery shopping!

**Objective Function**  
Minimise cost of grocery bill

**Contraints**  
Be within the lower and upper bounds for certain macronutrients (proteins, fats, carbs) and micronutrients (vitamins, minerals).  
Be within the lower and upper bounds for certain food amounts (eg. we don't want to eat more than 500g ofbroccoli).

**Disclaimer**  
It goes without saying that I'm not a medical professional, I just try my best to find reliable information sources (books, websites, youtube videos).

None of this should be taken as advice; I am merely documenting some of my own personal findings which has been conducted with my own personal goals in mind.

Nutritional science is a very young and extremely complicated science. Controlling for variables and getting reliable results in studies is absurdly difficult. New studies come out on a regular basis which can either open up new doors or disqualifty older information (I follow Dr. Rhonda Patrick for some of the more cutting-edge in health sciences).

**Practitioners vs Theorists**  
The effects of different macronutrient protocols are fairly measureable (weight, bodyfat %, gym strength, satiety)  and have a decently quick feedback loop (effects are seen in a couple of days to weeks). Because of this, I tend to follow practitioners for macronutrient advice eg. fitness trainers, bodybuilders, athletes.

The effects of micronutrients are quite subtle and tougher to measure (requires blood tests) and usually don't manifest themselves until perhaps months or years later. Because of this, I tend to follow theorists for micronutrient advice eg. researchers, academics.

Of course practitioners and theorists are not mutually exclusive, but some people tend to lean further one way than the other.

In [2]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize

Firstly, let's create a dataframe that contains our nutritional requirements.

Source of macronutrient recommendations:  
https://www.youtube.com/watch?v=kjGM8NSkqTU

Source of micronutrient recommendations:  
https://www.health.harvard.edu/staying-healthy/listing_of_vitamins

It should be worth noting that we won't be controlling for every conceivable nutrient because:
- Our parameter space will be absurd
- Data/information/recommendations for that specific nutrient doesn't exist (eg. phytonutrients)
- Some nutrients aren't efficient to obtain via food (eg. vitamin D, better to obtain from supplements/sun)
- KISS: practically speaking, humans have been doing alright without mathematically micro-managing our nutrition.

In [3]:
nutrient_rdas = pd.DataFrame()

nutrient_rdas.loc['Protein', 'Lower'] = 105
nutrient_rdas.loc['Protein', 'Upper'] = 125
nutrient_rdas.loc['Protein', 'Unit'] = 'g'

nutrient_rdas.loc['Fat', 'Lower'] = 50
nutrient_rdas.loc['Fat', 'Upper'] = 70
nutrient_rdas.loc['Fat', 'Unit'] = 'g'

nutrient_rdas.loc['Carb', 'Lower'] = 120
nutrient_rdas.loc['Carb', 'Upper'] = 240
nutrient_rdas.loc['Carb', 'Unit'] = 'g'

nutrient_rdas.loc['Fiber', 'Lower'] = 30
nutrient_rdas.loc['Fiber', 'Upper'] = 'unknown'
nutrient_rdas.loc['Fiber', 'Unit'] = 'g'

nutrient_rdas.loc['Energy', 'Lower'] = 1500
nutrient_rdas.loc['Energy', 'Upper'] = 1800
nutrient_rdas.loc['Energy', 'Unit'] = 'kcal'

# I am increasing the Harvard RDA upper limit due to:
# Quote: "Cronic vitamin A toxicity (excessive ingestion of 4000IU/kg for 6 to 15 months)"
# Source: https://www.who.int/immunization/programmes_systems/interventions/Adverse_events_vitA.pdf?ua=1

# 10000IU seems like an absurdly low upper limit, as 100g of carrots has around 16000IU
# Source: https://nutritiondata.self.com/facts/vegetables-and-vegetable-products/2383/2
nutrient_rdas.loc['VitaminA', 'Lower'] = 3000
nutrient_rdas.loc['VitaminA', 'Upper'] = 100000
nutrient_rdas.loc['VitaminA', 'Unit'] = 'IU'

# AKA Thiamin
nutrient_rdas.loc['VitaminB1', 'Lower'] = 1.2
nutrient_rdas.loc['VitaminB1', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminB1', 'Unit'] = 'mg'

# AKA Riboflavin
nutrient_rdas.loc['VitaminB2', 'Lower'] = 1.3
nutrient_rdas.loc['VitaminB2', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminB2', 'Unit'] = 'mg'

# AKA Niacin
# I am increasing the Harvard RDA upper limit due to:
# Quote: "According to the Mayo Clinic, serious side effects can occur at daily doses of 2,000 mg or higher."
# Source: https://www.healthline.com/health/food-nutrition/niacin-overdose#lethal-dosage

# Quote: " At much higher doses (1000 to 2000 mg per day) niacin is used as a treatment for high cholesterol."
# Source: http://www.dpic.org/article/professional/niacin-facts-flushing
nutrient_rdas.loc['VitaminB3', 'Lower'] = 16
nutrient_rdas.loc['VitaminB3', 'Upper'] = 500
nutrient_rdas.loc['VitaminB3', 'Unit'] = 'mg'

# AKA Pantothenic Acid
nutrient_rdas.loc['VitaminB5', 'Lower'] = 5
nutrient_rdas.loc['VitaminB5', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminB5', 'Unit'] = 'mg'

nutrient_rdas.loc['VitaminB6', 'Lower'] = 1.3
nutrient_rdas.loc['VitaminB6', 'Upper'] = 100
nutrient_rdas.loc['VitaminB6', 'Unit'] = 'mg'

# AKA Folate
# I am increasing the Harvard RDA upper limit due to:
# Quote: "The Institute of Medicine has also set a "tolerable upper limit" of 1,000 mcg per day of folic
#         acid from fortified foods or supplements; there's no limit on folate from food."
# Source: https://www.health.harvard.edu/newsletter_article/Folic_acid_Too_much_of_a_good_thing
nutrient_rdas.loc['VitaminB9', 'Lower'] = 400
nutrient_rdas.loc['VitaminB9', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminB9', 'Unit'] = 'mcg'

nutrient_rdas.loc['VitaminB12', 'Lower'] = 2.4
nutrient_rdas.loc['VitaminB12', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminB12', 'Unit'] = 'mcg'

nutrient_rdas.loc['VitaminC', 'Lower'] = 90
nutrient_rdas.loc['VitaminC', 'Upper'] = 2000
nutrient_rdas.loc['VitaminC', 'Unit'] = 'mg'

nutrient_rdas.loc['VitaminE', 'Lower'] = 15
nutrient_rdas.loc['VitaminE', 'Upper'] = 1000
nutrient_rdas.loc['VitaminE', 'Unit'] = 'mg'

nutrient_rdas.loc['VitaminK', 'Lower'] = 120
nutrient_rdas.loc['VitaminK', 'Upper'] = 'unknown'
nutrient_rdas.loc['VitaminK', 'Unit'] = 'mcg'

# Add some reasoning for choline reduction
nutrient_rdas.loc['Choline', 'Lower'] = 360
nutrient_rdas.loc['Choline', 'Upper'] = 3500
nutrient_rdas.loc['Choline', 'Unit'] = 'mg'

nutrient_rdas.loc['Calcium', 'Lower'] = 1000
nutrient_rdas.loc['Calcium', 'Upper'] = 2500
nutrient_rdas.loc['Calcium', 'Unit'] = 'mg'

nutrient_rdas.loc['Copper', 'Lower'] = 0.9
nutrient_rdas.loc['Copper', 'Upper'] = 10
nutrient_rdas.loc['Copper', 'Unit'] = 'mg'

nutrient_rdas.loc['Iron', 'Lower'] = 8
nutrient_rdas.loc['Iron', 'Upper'] = 45
nutrient_rdas.loc['Iron', 'Unit'] = 'mg'

nutrient_rdas.loc['Magnesium', 'Lower'] = 420
nutrient_rdas.loc['Magnesium', 'Upper'] = 'unknown'
nutrient_rdas.loc['Magnesium', 'Unit'] = 'mg'

nutrient_rdas.loc['Manganese', 'Lower'] = 2.3
nutrient_rdas.loc['Manganese', 'Upper'] = 11
nutrient_rdas.loc['Manganese', 'Unit'] = 'mg'

nutrient_rdas.loc['Phosphorus', 'Lower'] = 700
nutrient_rdas.loc['Phosphorus', 'Upper'] = 4000
nutrient_rdas.loc['Phosphorus', 'Unit'] = 'mg'

nutrient_rdas.loc['Potassium', 'Lower'] = 4700
nutrient_rdas.loc['Potassium', 'Upper'] = 'unknown'
nutrient_rdas.loc['Potassium', 'Unit'] = 'mg'

nutrient_rdas.loc['Selenium', 'Lower'] = 55
nutrient_rdas.loc['Selenium', 'Upper'] = 400
nutrient_rdas.loc['Selenium', 'Unit'] = 'mcg'

nutrient_rdas.loc['Zinc', 'Lower'] = 11
nutrient_rdas.loc['Zinc', 'Upper'] = 40
nutrient_rdas.loc['Zinc', 'Unit'] = 'mg'

In [4]:
nutrient_rdas

,Lower,Upper,Unit
Protein,105.0,125,g
Fat,50.0,70,g
Carb,120.0,240,g
Fiber,30.0,unknown,g
Energy,1500.0,1800,kcal
VitaminA,3000.0,100000,IU
VitaminB1,1.2,unknown,mg
VitaminB2,1.3,unknown,mg
VitaminB3,16.0,500,mg
VitaminB5,5.0,unknown,mg


*"The RDA (Recommended Dietary Allowance) covers more than 97% of the population...  
The RDA is based on the population for which the mean and standard deviation were determined. Thus, different populations (children, men, women, etc.) have different RDAs."*

Source: https://library.med.utah.edu/NetBiochem/nutrition/lect2/1_1.html

Because I am shorter and lighter than the average male, I will be (arbitrarily) reducing the lower limits for micronutrients (index 'VitaminA' and onwards)

In [5]:
nutrient_rdas.loc['VitaminA':, 'Lower'] = nutrient_rdas.loc['VitaminA':, 'Lower'] * 0.85

In [6]:
nutrient_rdas

,Lower,Upper,Unit
Protein,105.000,125,g
Fat,50.000,70,g
Carb,120.000,240,g
Fiber,30.000,unknown,g
Energy,1500.000,1800,kcal
VitaminA,2550.000,100000,IU
VitaminB1,1.020,unknown,mg
VitaminB2,1.105,unknown,mg
VitaminB3,13.600,500,mg
VitaminB5,4.250,unknown,mg


At this point, it may be tempting grab a huge dataset and brute force an optimal solution. However, we will be hand selecting a foods list for the following reasions:
- We want to minimise the parameter space due to limitations in computing power
- Realistically, only a handful of foods will lie in the set of likely solution spaces
- Data sources with detailed nutrition data don't have prices, data sources with prices don't have detailed nutrition data. A bit of manual work will be involved to merge the two

I will be following the rough guidelines when selecting foods and their nutritional content:
- Use my own prior knowedge of nutrient dense foods
- Google "nutrient dense foods" or "foods high in XYZ" to obtain a short list of foods to add to our parameter space
- Opt for plant-based options where possible
- Avoid exotic foods that are hard to find in regular supermarkets
- Use raw weights of foods to calculate nutritional values, as we buy foods in their raw weights, not cooked weights

Nutritional data of foods will be sourced from: https://nutritiondata.self.com/

In [7]:
nd = pd.read_csv('nutdata.csv').set_index('Item')

In [8]:
nd

,Price,Qty,Unit,Protein,Fat,Carb,Fiber,Energy,VitaminA,VitaminB1,...,Choline,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc
Item,,,,,,,,,,,,,,,,,,,,,
brazil nuts,3.60,100,g,14.3,66.4,12.3,7.5,689,0,0.6,...,28.8,160.0,1.70,2.4,376.0,1.20,725.0,659.0,1917.0,4.1
broccoli,0.50,100,g,2.8,0.3,4.8,3.0,26,1034,0.1,...,14.9,56.0,0.00,0.8,18.0,0.30,50.0,212.0,2.8,0.5
eggs,1.00,100,g,12.6,9.9,0.8,0.0,143,487,0.1,...,251.0,53.0,0.10,1.8,12.0,0.00,191.0,134.0,31.7,1.1
lentils,0.46,100,g,25.8,1.1,60.1,30.5,353,39,0.9,...,96.4,56.0,0.50,7.5,122.0,1.30,451.0,955.0,8.3,4.8
kale,0.40,100,g,2.8,0.5,5.2,2.0,30,14705,0.0,...,0.1,38.6,0.00,0.3,5.0,0.10,7.8,89.9,0.3,0.1
milk (full cream),0.13,100,ml,3.2,3.3,5.3,0.0,60,102,0.0,...,14.2,113.0,0.00,0.0,10.0,0.00,91.0,143.0,3.7,0.4
muesli,0.28,100,g,11.2,5.2,65.1,8.7,350,0,0.3,...,36.1,52.0,0.40,3.9,112.0,3.10,368.0,323.0,21.9,3.1
nutritional yeast,4.00,100,g,47.0,4.8,19.8,18.7,323,0,2.9,...,0.0,0.0,0.04,0.2,7.1,0.04,0.0,0.0,0.0,0.8
oil,1.20,100,ml,0.0,91.0,0.0,0.0,812,0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [9]:
# select the nutrients columns (ie. between 'Protein' and 'Zinc')
columns = nd.loc[:, 'Protein':].columns

Because we have so many constraints, a lot of code is required is set up all the constraints for our optimiser.  
Let's write to some code to help us write code.

The optimiser we're using requires inequality constraints to be in the form f(x) > 0  

Eg. the contraint Protein > 105 will be moved to Protein - 105 > 0, so f(x) = Protein - 105

In [10]:
for col in columns:
    s = """def cons_{}_{}(x):
    return np.dot(x, nd.{}) - {}
    """.format(col, 'Lower', col, 'nutrient_rdas.loc[\'{}\', \'Lower\']'.format(col))
    print(s)
    
    if nutrient_rdas.loc[col, 'Upper'] == 'unknown':
        continue
        
    s = """def cons_{}_{}(x):
    return {} - np.dot(x, nd.{})
    """.format(col, 'Upper', 'nutrient_rdas.loc[\'{}\', \'Upper\']'.format(col), col)
    print(s)

def cons_Protein_Lower(x):
    return np.dot(x, nd.Protein) - nutrient_rdas.loc['Protein', 'Lower']
    
def cons_Protein_Upper(x):
    return nutrient_rdas.loc['Protein', 'Upper'] - np.dot(x, nd.Protein)
    
def cons_Fat_Lower(x):
    return np.dot(x, nd.Fat) - nutrient_rdas.loc['Fat', 'Lower']
    
def cons_Fat_Upper(x):
    return nutrient_rdas.loc['Fat', 'Upper'] - np.dot(x, nd.Fat)
    
def cons_Carb_Lower(x):
    return np.dot(x, nd.Carb) - nutrient_rdas.loc['Carb', 'Lower']
    
def cons_Carb_Upper(x):
    return nutrient_rdas.loc['Carb', 'Upper'] - np.dot(x, nd.Carb)
    
def cons_Fiber_Lower(x):
    return np.dot(x, nd.Fiber) - nutrient_rdas.loc['Fiber', 'Lower']
    
def cons_Energy_Lower(x):
    return np.dot(x, nd.Energy) - nutrient_rdas.loc['Energy', 'Lower']
    
def cons_Energy_Upper(x):
    return nutrient_rdas.loc['Energy', 'Upper'] - np.dot(x, nd.Energy)
    
def cons_VitaminA_Lower(x):
    return np.dot(x, nd.VitaminA) - nutrient_rdas.loc['VitaminA', 'Lower']
    
d

We also need to tell our optimiser the type of constraint and what function it points to

In [11]:
for col in columns:
    s = """{{'type': 'ineq', 'fun': cons_{}_{}}},""".format(col, 'Lower')
    print(s)
    
    if nutrient_rdas.loc[col, 'Upper'] == 'unknown':
        continue
        
    s = """{{'type': 'ineq', 'fun': cons_{}_{}}},""".format(col, 'Upper')
    print(s)

{'type': 'ineq', 'fun': cons_Protein_Lower},
{'type': 'ineq', 'fun': cons_Protein_Upper},
{'type': 'ineq', 'fun': cons_Fat_Lower},
{'type': 'ineq', 'fun': cons_Fat_Upper},
{'type': 'ineq', 'fun': cons_Carb_Lower},
{'type': 'ineq', 'fun': cons_Carb_Upper},
{'type': 'ineq', 'fun': cons_Fiber_Lower},
{'type': 'ineq', 'fun': cons_Energy_Lower},
{'type': 'ineq', 'fun': cons_Energy_Upper},
{'type': 'ineq', 'fun': cons_VitaminA_Lower},
{'type': 'ineq', 'fun': cons_VitaminA_Upper},
{'type': 'ineq', 'fun': cons_VitaminB1_Lower},
{'type': 'ineq', 'fun': cons_VitaminB2_Lower},
{'type': 'ineq', 'fun': cons_VitaminB3_Lower},
{'type': 'ineq', 'fun': cons_VitaminB3_Upper},
{'type': 'ineq', 'fun': cons_VitaminB5_Lower},
{'type': 'ineq', 'fun': cons_VitaminB6_Lower},
{'type': 'ineq', 'fun': cons_VitaminB6_Upper},
{'type': 'ineq', 'fun': cons_VitaminB9_Lower},
{'type': 'ineq', 'fun': cons_VitaminB12_Lower},
{'type': 'ineq', 'fun': cons_VitaminC_Lower},
{'type': 'ineq', 'fun': cons_VitaminC_Upper},
{'typ

In [12]:
def cons_Protein_Lower(x):
    return np.dot(x, nd.Protein) - nutrient_rdas.loc['Protein', 'Lower']
    
def cons_Protein_Upper(x):
    return nutrient_rdas.loc['Protein', 'Upper'] - np.dot(x, nd.Protein)
    
def cons_Fat_Lower(x):
    return np.dot(x, nd.Fat) - nutrient_rdas.loc['Fat', 'Lower']
    
def cons_Fat_Upper(x):
    return nutrient_rdas.loc['Fat', 'Upper'] - np.dot(x, nd.Fat)
    
def cons_Carb_Lower(x):
    return np.dot(x, nd.Carb) - nutrient_rdas.loc['Carb', 'Lower']
    
def cons_Carb_Upper(x):
    return nutrient_rdas.loc['Carb', 'Upper'] - np.dot(x, nd.Carb)
    
def cons_Fiber_Lower(x):
    return np.dot(x, nd.Fiber) - nutrient_rdas.loc['Fiber', 'Lower']
    
def cons_Energy_Lower(x):
    return np.dot(x, nd.Energy) - nutrient_rdas.loc['Energy', 'Lower']
    
def cons_Energy_Upper(x):
    return nutrient_rdas.loc['Energy', 'Upper'] - np.dot(x, nd.Energy)
    
def cons_VitaminA_Lower(x):
    return np.dot(x, nd.VitaminA) - nutrient_rdas.loc['VitaminA', 'Lower']
    
def cons_VitaminA_Upper(x):
    return nutrient_rdas.loc['VitaminA', 'Upper'] - np.dot(x, nd.VitaminA)
    
def cons_VitaminB1_Lower(x):
    return np.dot(x, nd.VitaminB1) - nutrient_rdas.loc['VitaminB1', 'Lower']
    
def cons_VitaminB2_Lower(x):
    return np.dot(x, nd.VitaminB2) - nutrient_rdas.loc['VitaminB2', 'Lower']
    
def cons_VitaminB3_Lower(x):
    return np.dot(x, nd.VitaminB3) - nutrient_rdas.loc['VitaminB3', 'Lower']
    
def cons_VitaminB3_Upper(x):
    return nutrient_rdas.loc['VitaminB3', 'Upper'] - np.dot(x, nd.VitaminB3)
    
def cons_VitaminB5_Lower(x):
    return np.dot(x, nd.VitaminB5) - nutrient_rdas.loc['VitaminB5', 'Lower']
    
def cons_VitaminB6_Lower(x):
    return np.dot(x, nd.VitaminB6) - nutrient_rdas.loc['VitaminB6', 'Lower']
    
def cons_VitaminB6_Upper(x):
    return nutrient_rdas.loc['VitaminB6', 'Upper'] - np.dot(x, nd.VitaminB6)
    
def cons_VitaminB9_Lower(x):
    return np.dot(x, nd.VitaminB9) - nutrient_rdas.loc['VitaminB9', 'Lower']
    
def cons_VitaminB12_Lower(x):
    return np.dot(x, nd.VitaminB12) - nutrient_rdas.loc['VitaminB12', 'Lower']
    
def cons_VitaminC_Lower(x):
    return np.dot(x, nd.VitaminC) - nutrient_rdas.loc['VitaminC', 'Lower']
    
def cons_VitaminC_Upper(x):
    return nutrient_rdas.loc['VitaminC', 'Upper'] - np.dot(x, nd.VitaminC)
    
def cons_VitaminE_Lower(x):
    return np.dot(x, nd.VitaminE) - nutrient_rdas.loc['VitaminE', 'Lower']
    
def cons_VitaminE_Upper(x):
    return nutrient_rdas.loc['VitaminE', 'Upper'] - np.dot(x, nd.VitaminE)
    
def cons_VitaminK_Lower(x):
    return np.dot(x, nd.VitaminK) - nutrient_rdas.loc['VitaminK', 'Lower']
    
def cons_Choline_Lower(x):
    return np.dot(x, nd.Choline) - nutrient_rdas.loc['Choline', 'Lower']
    
def cons_Choline_Upper(x):
    return nutrient_rdas.loc['Choline', 'Upper'] - np.dot(x, nd.Choline)
    
def cons_Calcium_Lower(x):
    return np.dot(x, nd.Calcium) - nutrient_rdas.loc['Calcium', 'Lower']
    
def cons_Calcium_Upper(x):
    return nutrient_rdas.loc['Calcium', 'Upper'] - np.dot(x, nd.Calcium)
    
def cons_Copper_Lower(x):
    return np.dot(x, nd.Copper) - nutrient_rdas.loc['Copper', 'Lower']
    
def cons_Copper_Upper(x):
    return nutrient_rdas.loc['Copper', 'Upper'] - np.dot(x, nd.Copper)
    
def cons_Iron_Lower(x):
    return np.dot(x, nd.Iron) - nutrient_rdas.loc['Iron', 'Lower']
    
def cons_Iron_Upper(x):
    return nutrient_rdas.loc['Iron', 'Upper'] - np.dot(x, nd.Iron)
    
def cons_Magnesium_Lower(x):
    return np.dot(x, nd.Magnesium) - nutrient_rdas.loc['Magnesium', 'Lower']
    
def cons_Manganese_Lower(x):
    return np.dot(x, nd.Manganese) - nutrient_rdas.loc['Manganese', 'Lower']
    
def cons_Manganese_Upper(x):
    return nutrient_rdas.loc['Manganese', 'Upper'] - np.dot(x, nd.Manganese)
    
def cons_Phosphorus_Lower(x):
    return np.dot(x, nd.Phosphorus) - nutrient_rdas.loc['Phosphorus', 'Lower']
    
def cons_Phosphorus_Upper(x):
    return nutrient_rdas.loc['Phosphorus', 'Upper'] - np.dot(x, nd.Phosphorus)
    
def cons_Potassium_Lower(x):
    return np.dot(x, nd.Potassium) - nutrient_rdas.loc['Potassium', 'Lower']
    
def cons_Selenium_Lower(x):
    return np.dot(x, nd.Selenium) - nutrient_rdas.loc['Selenium', 'Lower']
    
def cons_Selenium_Upper(x):
    return nutrient_rdas.loc['Selenium', 'Upper'] - np.dot(x, nd.Selenium)
    
def cons_Zinc_Lower(x):
    return np.dot(x, nd.Zinc) - nutrient_rdas.loc['Zinc', 'Lower']
    
def cons_Zinc_Upper(x):
    return nutrient_rdas.loc['Zinc', 'Upper'] - np.dot(x, nd.Zinc)

In [13]:
constraints = [    
    {'type': 'ineq', 'fun': cons_Protein_Lower},
    {'type': 'ineq', 'fun': cons_Protein_Upper},
    {'type': 'ineq', 'fun': cons_Fat_Lower},
    {'type': 'ineq', 'fun': cons_Fat_Upper},
    {'type': 'ineq', 'fun': cons_Carb_Lower},
    {'type': 'ineq', 'fun': cons_Carb_Upper},
    {'type': 'ineq', 'fun': cons_Fiber_Lower},
    {'type': 'ineq', 'fun': cons_Energy_Lower},
    {'type': 'ineq', 'fun': cons_Energy_Upper},
    {'type': 'ineq', 'fun': cons_VitaminA_Lower},
    {'type': 'ineq', 'fun': cons_VitaminA_Upper},
    {'type': 'ineq', 'fun': cons_VitaminB1_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB2_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB3_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB3_Upper},
    {'type': 'ineq', 'fun': cons_VitaminB5_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB6_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB6_Upper},
    {'type': 'ineq', 'fun': cons_VitaminB9_Lower},
    {'type': 'ineq', 'fun': cons_VitaminB12_Lower},
    {'type': 'ineq', 'fun': cons_VitaminC_Lower},
    {'type': 'ineq', 'fun': cons_VitaminC_Upper},
    {'type': 'ineq', 'fun': cons_VitaminE_Lower},
    {'type': 'ineq', 'fun': cons_VitaminE_Upper},
    {'type': 'ineq', 'fun': cons_VitaminK_Lower},
    {'type': 'ineq', 'fun': cons_Choline_Lower},
    {'type': 'ineq', 'fun': cons_Choline_Upper},
    {'type': 'ineq', 'fun': cons_Calcium_Lower},
    {'type': 'ineq', 'fun': cons_Calcium_Upper},
    {'type': 'ineq', 'fun': cons_Copper_Lower},
    {'type': 'ineq', 'fun': cons_Copper_Upper},
    {'type': 'ineq', 'fun': cons_Iron_Lower},
    {'type': 'ineq', 'fun': cons_Iron_Upper},
    {'type': 'ineq', 'fun': cons_Magnesium_Lower},
    {'type': 'ineq', 'fun': cons_Manganese_Lower},
    {'type': 'ineq', 'fun': cons_Manganese_Upper},
    {'type': 'ineq', 'fun': cons_Phosphorus_Lower},
    {'type': 'ineq', 'fun': cons_Phosphorus_Upper},
    {'type': 'ineq', 'fun': cons_Potassium_Lower},
    {'type': 'ineq', 'fun': cons_Selenium_Lower},
    {'type': 'ineq', 'fun': cons_Selenium_Upper},
    {'type': 'ineq', 'fun': cons_Zinc_Lower},
    {'type': 'ineq', 'fun': cons_Zinc_Upper},
]

In [14]:
def obj_fnc(x):
    return np.dot(x, nd.Price)

We should also set lower and upper limit of how many units of each food we can buy.  
x's base unit is 100 times 'Unit' eg. x = 0.5 for brazil nuts implies 50g of brazil nuts

In [15]:
qty_nd = nd.loc[:, :'Unit']

# default x bound values
qty_nd['x_lower'] = 0.0
qty_nd['x_upper'] = 5.0 # don't want to eat more than 500g or something per day

In [30]:
# I want to use at least 1 scoop of protein per day so it doesn't expire and go to waste
food_name = 'whey protein'
bound = 'x_lower'
value = 0.3
qty_nd.loc[food_name, bound] = value

In [31]:
qty_nd

,Price,Qty,Unit,x_lower,x_upper,x,x_g
Item,,,,,,,
brazil nuts,3.60,100,g,0.0,5.0,0.0032,0.32
broccoli,0.50,100,g,0.0,5.0,1.0560,105.60
eggs,1.00,100,g,0.0,5.0,0.0308,3.08
lentils,0.46,100,g,0.0,5.0,0.5412,54.12
kale,0.40,100,g,0.0,5.0,0.0000,0.00
milk (full cream),0.13,100,ml,0.0,5.0,5.0000,500.00
muesli,0.28,100,g,0.0,5.0,0.0000,0.00
nutritional yeast,4.00,100,g,0.0,5.0,0.0000,0.00
oil,1.20,100,ml,0.0,5.0,0.0000,0.00


In [32]:
# set up the list of bound tuples to pass to our optimiser

bounds = []

for i, row in qty_nd.iterrows():
    bound_tuple = (row['x_lower'], row['x_upper'])
    bounds.append(bound_tuple)

In [33]:
# initialise our x vector with arbitrary values as a starting point for the optimiser
x = np.full(len(nd), 0.0)

In [34]:
sol = minimize(obj_fnc, x, method='SLSQP', bounds=bounds, constraints=constraints, tol=0.01)

In [35]:
sol

     fun: 3.134879743103258
     jac: array([3.59999996, 0.5       , 1.        , 0.45999998, 0.39999998,
       0.13      , 0.27999997, 4.        , 1.19999999, 0.47999996,
       0.19999999, 1.59999996, 1.59999996, 0.38      , 0.32999998,
       0.69999999, 0.5       , 2.        ])
 message: 'Optimization terminated successfully.'
    nfev: 180
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([2.76032074e-03, 1.24439230e+00, 3.07692308e-02, 2.88632957e-02,
       0.00000000e+00, 5.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.17524753e-01, 5.37965447e-02, 0.00000000e+00,
       0.00000000e+00, 1.85786685e-02, 2.66656582e+00, 2.26431839e-01,
       0.00000000e+00, 3.00000000e-01])

In [36]:
xsol = pd.Series(sol.x, index=nd.index)

In [37]:
qty_nd['x'] = xsol.round(4)
qty_nd['x_g'] = (100 * xsol).round(2)

In [38]:
# let's have a look out our shopping list

qty_nd.loc[qty_nd['x'] > 0.001]

,Price,Qty,Unit,x_lower,x_upper,x,x_g
Item,,,,,,,
brazil nuts,3.60,100,g,0.0,5.0,0.0028,0.28
broccoli,0.50,100,g,0.0,5.0,1.2444,124.44
eggs,1.00,100,g,0.0,5.0,0.0308,3.08
lentils,0.46,100,g,0.0,5.0,0.0289,2.89
milk (full cream),0.13,100,ml,0.0,5.0,5.0000,500.00
peanut butter,0.48,100,g,0.0,5.0,0.3175,31.75
peas,0.20,100,g,0.0,5.0,0.0538,5.38
spinach,0.38,100,g,0.0,5.0,0.0186,1.86
split peas,0.33,100,g,0.0,5.0,2.6666,266.66


In [39]:
# this is how much is would cost for a day's worth of nutrients

nutrient_rdas['xsol_nutr'] = pd.Series(np.dot(xsol, nd.loc[:, 'Protein':]).round(2), index=nutrient_rdas.index)
print('Cost:', obj_fnc(xsol))

Cost: 3.134879743103258


Not bad! As expected with mathematical optimisation, it's expecting us to eat unreasonably small amounts of some foods.  
So I'm gonna cherry pick a couple of foods from that list and conveniently pick nicer numbers that are realistic for a shopping trip.

In [40]:
msol = pd.Series(data=[0.0 for x in range(len(xsol))], index=xsol.index)

msol['broccoli'] = 1.25
msol['milk (full cream)'] = 5
msol['peanut butter'] = 0.3
msol['spinach'] = 0.6
msol['split peas'] = 2
msol['sunflower kernels'] = 0.3
msol['whey protein'] = 0.3

# msol = pd.Series(data=[0.0 for x in range(len(xsol))], index=xsol.index)

# msol['broccoli'] = 1
# msol['lentils'] = 0.5
# msol['milk (full cream)'] = 5
# msol['peanut butter'] = 0.8
# msol['peas'] = 0.5
# msol['spinach'] = 0.3
# msol['split peas'] = 1.8
# msol['sunflower kernels'] = 0.1

In [41]:
nutrient_rdas['msol_nutr'] = pd.Series(np.dot(msol, nd.loc[:, 'Protein':]).round(2), index=nutrient_rdas.index)
print('Cost:', obj_fnc(msol).round(2))

Cost: 3.12


In [42]:
nutrient_rdas['deficit%'] = np.where(nutrient_rdas['msol_nutr'] < nutrient_rdas['Lower'], 100 * abs(nutrient_rdas['msol_nutr'] - nutrient_rdas['Lower']) / nutrient_rdas['Lower'], 0).round(2)

In [43]:
nutrient_rdas

,Lower,Upper,Unit,xsol_nutr,msol_nutr,deficit%
Protein,105.000,125,g,122.83,108.18,0.00
Fat,50.000,70,g,50.00,51.91,0.00
Carb,120.000,240,g,208.82,169.62,0.00
Fiber,30.000,unknown,g,76.78,60.87,0.00
Energy,1500.000,1800,kcal,1719.75,1523.00,0.00
VitaminA,2550.000,100000,IU,2550.00,9150.50,0.00
VitaminB1,1.020,unknown,mg,2.41,2.06,0.00
VitaminB2,1.105,unknown,mg,1.84,2.88,0.00
VitaminB3,13.600,500,mg,14.92,13.48,0.88
VitaminB5,4.250,unknown,mg,7.90,6.70,0.00


In [88]:
df = pd.DataFrame()
df['AmtToBuy'] = msol

df = pd.merge(df, nd, left_index=True, right_index=True)

foods = df[df['AmtToBuy'] > 0.1]
rdas = nutrient_rdas[['Lower', 'Upper', 'Unit']]

In [89]:
foods

,AmtToBuy,Price,Qty,Unit,Protein,Fat,Carb,Fiber,Energy,VitaminA,...,Choline,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc
Item,,,,,,,,,,,,,,,,,,,,,
broccoli,1.66,0.50,100,g,2.8,0.3,4.8,3.0,26,1034,...,14.9,56.0,0.0,0.8,18.0,0.3,50.0,212.0,2.8,0.5
milk (full cream),5.00,0.13,100,ml,3.2,3.3,5.3,0.0,60,102,...,14.2,113.0,0.0,0.0,10.0,0.0,91.0,143.0,3.7,0.4
peanut butter,0.30,0.48,100,g,25.1,50.4,20.0,6.0,619,0,...,63.0,43.0,0.5,1.9,154.0,1.5,358.0,649.0,5.6,2.9
spinach,0.40,0.38,100,g,3.6,0.6,4.2,2.9,29,11725,...,22.1,129.0,0.1,1.9,75.0,0.7,49.0,346.0,6.0,0.6
split peas,2.00,0.33,100,g,24.6,1.2,60.4,25.5,341,149,...,95.5,55.0,0.9,4.4,115.0,1.4,366.0,981.0,1.6,3.0
sunflower kernels,0.30,0.70,100,g,20.8,51.5,20.0,8.6,612,50,...,55.1,78.0,1.8,5.2,325.0,1.9,660.0,645.0,53.0,5.0
whey protein,0.30,2.00,100,g,78.5,5.7,6.0,0.0,406,0,...,0.0,350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
rdas

,Lower,Upper,Unit
Protein,105.000,125,g
Fat,50.000,70,g
Carb,120.000,240,g
Fiber,30.000,unknown,g
Energy,1500.000,1800,kcal
VitaminA,2550.000,100000,IU
VitaminB1,1.020,unknown,mg
VitaminB2,1.105,unknown,mg
VitaminB3,13.600,500,mg
VitaminB5,4.250,unknown,mg


In [91]:
for col in foods.loc[:, 'Protein':].columns:
    print(col, np.dot(foods[col], foods['AmtToBuy']))

Protein 108.60799999999999
Fat 51.91799999999999
Carb 170.748
Fiber 61.519999999999996
Energy 1527.8600000000001
VitaminA 7229.44
VitaminB1 2.086
VitaminB2 2.516
VitaminB3 13.508
VitaminB5 6.798
VitaminB6 1.1860000000000002
VitaminB9 832.5200000000001
VitaminB12 2.0
VitaminC 99.844
VitaminE 16.512
VitaminK 313.606
Choline 331.004
Calcium 960.8599999999999
Copper 2.53
Iron 13.018
Magnesium 483.58
Manganese 4.597999999999999
Phosphorus 1595.0
Potassium 3555.52
Selenium 46.327999999999996
Zinc 11.44
